In [1]:
from numpy.random import seed
seed(1)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from analysis import *
from collections import namedtuple
import Metrics
from PatientSet import PatientSet
from Constants import Constants
from Clustering import *
import re

#sklearn dependencies
from sklearn.metrics import roc_auc_score, adjusted_rand_score
from sklearn.utils import resample
from sklearn.cluster import FeatureAgglomeration

#we get like a million deprication errors for some reason with the external libraries
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Using TensorFlow backend.


In [2]:
#load in the patientset object that has all the patient info
db = PatientSet()

#add a bunch of features to the object that we'll want to try
#so we can use the db.to_dataframe function to get them all in a nice dataframe with one-hot encoding and labels automatically
db.t_volumes = np.array([np.sum([g.volume for g in gtvs]) for gtvs in db.gtvs]).reshape(-1,1)
db.bilateral = db.lateralities == 'B'
db.total_volumes = db.volumes.sum(axis = 1)
db.toxicity = db.feeding_tubes + db.aspiration > 0
db.tsimdoses = tsim_prediction(db)

D:\git_repos\CAMP-RT\PYTHON\Patient.py:360: RuntimeWarning: invalid value encountered in true_divide
  mean_tumor_distances /= tumor_volume
D:\git_repos\CAMP-RT\PYTHON\Patient.py:361: RuntimeWarning: invalid value encountered in true_divide
  tumor_position /= tumor_volume


error reading tumor volume for  10091
error reading tumor volume for  10148
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.

patient data loaded...



In [8]:
%load_ext autoreload
%autoreload 2
from Clustering import *

#parameters for the experiments
toxicities_to_test = ['toxicity']

#features to test the feature selection on.  should be fields in the patientset we have
#we don't cluster on these
unclusterable_features = ['t_volumes', 'bilateral', 'total_volumes']
#we cluster on these (each individually) if feature_clustering is defined,
clusterable_features = ['tumor_distances', 'volumes']

#features specifically for feature selection vs actually using.  Should either be
#some combo of actual and predicted dose for this
train_features = ['tsimdoses']
test_features = ['tsimdoses']

#number of times to resample and doing feature selection
#if n = 1, just use the first result
n_samples = 1

rescale = False
df_rescale = lambda d: (d - d.min(axis = 0))/(d.max(axis =0) - d.min(axis=0))

#put some bounds on the features to subset
min_features = 2

#for now just constrain it to one cluster
n_clusters = 2
selection_clusterer  = FClusterer(n_clusters)
cluster_feature_selector = FeatureClusterSelector(
    n_samples = n_samples,
    clusterer = selection_clusterer)
lg_feature_selector = FeatureSelector()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
train, test = get_train_test_datasets(db, 
                                      unclusterable_features, 
                                      clusterable_features, 
                                      train_features, 
                                      test_features)

In [9]:
cluster_feature_selector.get_importances(train, db.toxicity)

array([-6.84424679e-01, -2.63033305e-01, -8.28377745e-03, -5.94999771e-03,
       -1.08455090e-05, -3.14771006e-05, -1.98120856e-03, -2.83160105e-03,
       -7.73888742e-06, -5.41323721e-03, -5.38752248e-05, -7.66234536e-03,
       -4.34686532e-05, -3.82926075e-01, -1.85446881e-07, -7.85091124e-02,
       -1.77956239e-02, -2.74901224e-07, -1.27759882e-04, -6.02276399e-04,
       -2.41224386e-03, -5.85470991e-02, -5.66553235e-07, -1.00000000e+00,
       -3.92130261e-08, -1.17068526e-06, -4.76767750e-07, -9.42889940e-05,
       -6.13207582e-10, -1.00000000e+00, -1.37392302e-02, -1.70000000e-01,
       -3.93058762e-03, -1.00000000e+00, -7.25152782e-03, -1.00000000e+00,
       -2.11245609e-02, -1.20012910e-01, -3.15481572e-01, -3.56053660e-03,
       -1.00000000e+00, -1.34050749e-01, -2.67028737e-02, -8.05332527e-02,
       -1.33602233e-01, -9.88046219e-03, -2.81245087e-02, -5.70156405e-05,
       -8.39371395e-02, -1.48848092e-02, -1.56263805e-04, -4.95549825e-02,
       -4.24160941e-03, -